In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
 #!rm -rf /content/food\ food_

In [ ]:
import zipfile
zf = zipfile.ZipFile('gdrive/My Drive/food.zip', "r")
zf.extractall()

In [ ]:
# Каталог с данными для обучения
train_dir = './food/train/'
# Каталог с данными для проверки
val_dir = './food/val/'
# Каталог с данными для тестирования
test_dir = './food/test/'
# Размеры изображения
img_width, img_height = 250, 250
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 100
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 3000
# Количество изображений для проверки
nb_validation_samples = 500
# Количество изображений для тестирования
nb_test_samples = 1000

In [ ]:
_net = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(250, 250, 3))
_net.trainable = False
_net.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 124, 124, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 124, 124, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_205 (Activation)     (None, 124, 124, 32) 0           batch_normalization_204[0][0]    
________________________________________________________________________________

In [ ]:
model = Sequential()
model.add(_net)
model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rotation_range=180,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            rescale=1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 5 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 500 images belonging to 5 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1000 images belonging to 5 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=64,
    epochs=epochs,
    verbose=1,
    validation_data=val_generator,
    validation_steps=64)

Epoch 1/100
64/64 [==============================] - 51s 792ms/step - loss: 1.7057 - acc: 0.2119 - val_loss: 1.5368 - val_acc: 0.3470
Epoch 2/100
64/64 [==============================] - 32s 498ms/step - loss: 1.6099 - acc: 0.2461 - val_loss: 1.4189 - val_acc: 0.4680
Epoch 3/100
64/64 [==============================] - 32s 495ms/step - loss: 1.5696 - acc: 0.2900 - val_loss: 1.3617 - val_acc: 0.5650
Epoch 4/100
64/64 [==============================] - 32s 493ms/step - loss: 1.5121 - acc: 0.3643 - val_loss: 1.2574 - val_acc: 0.6260
Epoch 5/100
64/64 [==============================] - 31s 488ms/step - loss: 1.4609 - acc: 0.4062 - val_loss: 1.0850 - val_acc: 0.6710
Epoch 6/100
64/64 [==============================] - 32s 496ms/step - loss: 1.3808 - acc: 0.4404 - val_loss: 0.9265 - val_acc: 0.7040
Epoch 7/100
64/64 [==============================] - 32s 497ms/step - loss: 1.3014 - acc: 0.4912 - val_loss: 0.7779 - val_acc: 0.7440
Epoch 8/100
64/64 [==============================] - 31s 487ms

In [ ]:
scores_test = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных: %.2f%%" % (scores_test[1]*100))

Точность на тестовых данных: 89.82%


In [ ]:
scores_val = model.evaluate_generator(val_generator, nb_validation_samples // batch_size)
print("Точность на валидационных данных: %.2f%%" % (scores_val[1]*100))

Точность на валидационных данных: 88.51%


In [ ]:
scores_train = model.evaluate_generator(train_generator, nb_train_samples // batch_size)
print("Точность на тренировочных данных: %.2f%%" % (scores_train[1]*100))

Точность на тренировочных данных: 90.44%


In [ ]:
# scores

In [ ]:
import skimage
import numpy as np

picture = np.array([skimage.transform.resize(skimage.io.imread("test_pizza.jpg"), (250, 250))])
np.argmax(model.predict(picture))

pred = model.predict(picture)

In [ ]:
pred.max()

0.8870375

In [ ]:
res = pred.argmax(axis=1)

if (res == 0):
   print('Это крылья')
if (res == 1):
   print('Это пицца')
if (res == 2):
   print('Это салат')
if (res == 3):
   print('Это карбонара')
if (res == 4):
   print('Это вафли')


Это пицца


In [ ]:
pred

array([[5.4538250e-06, 8.8703752e-01, 2.6792288e-05, 1.3509393e-04,
        4.3286882e-06]], dtype=float32)

In [ ]:
# Run this cell to mount your Google Drive.
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
from joblib import dump

dump(model, 'food_rec_model.joblib')

['food_rec_model.joblib']